In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tdastro.astro_utils.passbands import PassbandGroup
from tdastro.sources.lightcurve_source import LightcurveModel

We start be loading the passbands that we will use to define the model. In this case we use the passbands from the LSST preset.

In [ ]:
passband_group = PassbandGroup(preset="LSST")
filters = passband_group.passbands.keys()
print(passband_group)

wavelengths = passband_group.waves
min_wave, max_wave = passband_group.wave_bounds()
print(f"Wavelengths range [{min_wave}, {max_wave}]")

passband_group.plot()

Next we want to create light curves for this source in each of the bands. This is what we want the model to reproduce then we call get_band_fluxes().

For simplicity of the demo, we create each curve as randomly parameterized sin waves.

In [ ]:
num_times = 100
times = np.linspace(0, 20, num_times)

lightcurves = {}
for filter in filters:
    amp = 5.0 * np.random.random() + 1.0
    flux_offset = np.random.random() * 25 + 10
    time_offset = np.random.random() * 10
    filter_flux = amp * np.sin(times + time_offset) + flux_offset
    print(f"Filter {filter}: {amp:.2f} * sin(t + {time_offset:.2f}) + {flux_offset:.2f}")

    lightcurves[filter] = np.array([times, filter_flux]).T

In [ ]:
# Plot the lightcurves
filter_plot_colors = {
    "LSST_u": "purple",
    "LSST_g": "blue",
    "LSST_r": "green",
    "LSST_i": "yellow",
    "LSST_z": "orange",
    "LSST_y": "red",
}

figure = plt.figure()
ax = figure.add_axes([0, 0, 1, 1])
for filter, lightcurve in lightcurves.items():
    color = filter_plot_colors.get(filter, "black")
    ax.plot(lightcurve[:, 0], lightcurve[:, 1], color=color, label=filter)

In [ ]:
model = LightcurveModel(
    lightcurves,
    passband_group,
)

If we plot the underlying lightcurves we can see they matched the provided ones.

In [ ]:
model.plot_lightcurves()

We can also plot the SEDs for each filter to see how the underlying model is computing the total SED. Each passband's SED basis is multiplied by the corresponding lightcurve's values over time to give its contributions to the overall SED.  Note that, in order to avoid overcounting the contributions of some wavelengths, the SED basis functions only contain wavelengths where the filters do not overlap.

In [ ]:
model.plot_sed_basis()